In [33]:
%reset -f

In [1]:
# [ ----- BASIC INSTALLS AND MODULE LOADING(S) ----- ]
!pip install sentence-transformers
!pip install tiktoken
# !pip install datasets
# pip install faiss-gpu-cu12
# !pip install pdfplumber
!pip install tqdm
import numpy as np
import torch
import sentence_transformers
# import datasets
import pandas
import json
from tqdm import tqdm

TRUE = True
FALSE = False
FRESH_START = FALSE

In [62]:
# [ ----- GET DATA TO BUILD RAG AROUND ----- ]
# this cell if for loading kaggle and getting BNS dataset
if FRESH_START:
  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  !kaggle datasets download -d nandr39/bharatiya-nyaya-sanhita-dataset-bns
  !unzip bharatiya-nyaya-sanhita-dataset-bns.zip -d bns_data


In [63]:
# [ ----- GET DATA TO BUILD RAG AROUND ----- ]
# this cell is for {loading kaggle} and supereme courts' dataset

if FRESH_START:
  #!kaggle datasets download -d adarshsingh0903/legal-dataset-sc-judgments-india-19502024
  #!unzip legal-dataset-sc-judgments-india-19502024.zip -d sc_data
    # IT DOESN'T WORK T-T use downloaded version uploaded on drive instead
  !cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/archive.zip /content/
  !unzip archive.zip -d /content/sc_judgments
  ### !cp -r /content/sc_judgments /content/drive/MyDrive/colab_stuff/court_hearings_rag/sc_judgments


In [64]:
# [----- DATA EXPLORATION, NO USE OF THIS CELL LATER ----- ]
"""
with pdfplumber.open("/content/sc_judgments/supreme_court_judgments/2020/Abhilasha_vs_Parkash_on_15_September_2020_1.PDF") as f:
  i = 0
  for page in f.pages:
    print(page.extract_text())
    print("\n\n")
    i += 1
    if i == 3:
      break
This confirms at on every page, no matter what year, the first line is the header and last the footer, we
henceforth remove them both. Though with time, minor details change like old ones have JUDGEMENT as heading
some new ones have J U D G E M E N T
"""

'\nwith pdfplumber.open("/content/sc_judgments/supreme_court_judgments/2020/Abhilasha_vs_Parkash_on_15_September_2020_1.PDF") as f:\n  i = 0\n  for page in f.pages:\n    print(page.extract_text())\n    print("\n\n")\n    i += 1\n    if i == 3:\n      break\nThis confirms at on every page, no matter what year, the first line is the header and last the footer, we\nhenceforth remove them both. Though with time, minor details change like old ones have JUDGEMENT as heading\nsome new ones have J U D G E M E N T\n'

In [88]:
# [----- To be fair, no use of this, just keeping it cuz it was used in earlier iterations ----- ]
# TODO :: REMOVE THIS
def strip_headers_footers(pages):
    cleaned_pages = []

    for i, page in enumerate(pages):
        # i need to remove header and footer from all pages keeping all other data intact
        cleaned_pages.append(page)

    return "".join(cleaned_pages)

In [89]:
# [ ------ For Saving Supereme courts' dataset cuz its too big to load everytime ----- ]
# we save each case data like this for later convinience, called in single core one
path_to_json = '/content/apex.jsonl' # advice gotten use json line, advice accepted
all_cases = []

def save_it(pages, file, file2, id):
  temp = {
      'id' : id,
      'source' : "Supreme Court",
      'title' : file2,
      'text' : strip_headers_footers(pages),
      'meta' : {
          'year' : file
          }
  }
  with open(path_to_json, "a", encoding='utf-8') as f:
    f.write(json.dumps(temp, ensure_ascii=False)) # dumps returns string of what dump was supposed to save
                                                  # we need to write line by line hence the way of f.write(json.dumps())
    f.write("\n") # this is how we make jsonl mate
all_cases.clear()

In [6]:
!cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl {path_to_json}


In [90]:
# [ ----- Responsible for removing headers and footers' of pdf of supereme court judgements ----- ]
def _apex_remove_header_footer(text):
  HEADER = None
  FOOTER = f'Indian Kanoon - '
  HEADER, _ = current_case['text'].split('\n', 1) # in the text region the first line before '\n' gives us header, so only 1 split needed
  cleaned_current_case = []

  for line in current_case['text'].split('\n'):
    if line.startswith(HEADER) or line.startswith(FOOTER):
      continue
    else:
      cleaned_current_case.append(line)
    # a weird observation, even after loads() my footer of this and header of next page don't have \n in between
    # so like just detecting footers is enough to remove headers as well
  return '\n'.join(cleaned_current_case)  # we split on '\n' so we join on '\n\

In [91]:
# [ ----- For cleaning subtle things bcs pdf has page so words get cut ----- ]
# OBSERVATION 1 - if a sentance ends with \n but no fullstop before it, it was cut midway, remove \n
# OBSERVATION 2 - if a word has -\n it was cut midway, remove -\n
# OBSERVATION 3 - multiple -- serve some purpose, wait now, remove later
def _remove_enter_mid_words(text):
  formatted_sentance = []
  sentances = text.split('. ')
  for sentance in sentances:
    sentance = sentance.replace('-\n', '')
    sentance = sentance.replace('\n', ' ')
    formatted_sentance.append(sentance)
  return '.\n'.join(formatted_sentance)

In [92]:
'''SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE'''
# [OH BOY! WHAT HAVE I GOTTEN MYSELF INTO ?!]
import os
apex_raw = []

if FRESH_START:
  # import pdfplumber
  ROOT = "/content/sc_judgments/supreme_court_judgments"
  start_id = sum(1 for _ in open(path_to_json, "r", encoding="utf-8"))
  last_line = None

  print(f'starting fro i :: {start_id}')
  i = 1
  for file in sorted(os.listdir(ROOT)): # order of traversal is weird without sorted
    # print(file)
    print(f"\ncurrently in year :: {file}\n currently in case :: ", end=' ') # so i don't die waiting
    for file2 in sorted(os.listdir(f"{ROOT}/{file}")):
      if i < start_id:
        i += 1
        continue

      print(f"{i}", sep=' ', end=' ')
      try:
        with pdfplumber.open(f"{ROOT}/{file}/{file2}") as pdf:
          pages = []
          for page in pdf.pages:
            try:
              text = page.extract_text()
              if text:
                pages.append(text)
                # print(pages)
            except Exception:
              print(f'something wrong with page in pdf :: {i} ')
              with open('/content/errors.jsonl', 'a') as f:
                f.write(json.dumps(f' PAGE PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
                f.write('\n')
                f.close()
          # only after pages has been appended with all data
          save_it(pages, file, file2, i)
          i += 1
      except Exception as e: # are these always meant to be used or always realized the need for
        print(f'something wrong with pdf :: {i}')
        with open('/content/errors.jsonl', 'a') as f:
          f.write(json.dumps(f' PDF PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
          f.write('\n')
          f.close()
    !cp {path_to_json} /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl
    print(f'\n:: copied to drive ::\n')
else:
  with open(path_to_json, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
      # remove header and footer; we get header in first page first line
      # each line read here is a case in itself cuz jsonl
      current_case = json.loads(line)
      current_case['text'] = _remove_enter_mid_words(_apex_remove_header_footer(current_case['text']))
      apex_raw.append(current_case)
      ### ONLY 100 SAMPLES FOR NOW
      # if i == 100:
      #   print(f"broken at i = {i}")
      #   break
#print(apex_raw)
#print(len(apex_raw))
print(apex_raw[0]['text'][:7000])

A.K.
Gopalan vs The State Of Madras.Union Of India: ...
on 19 May, 1950 Equivalent citations: 1950 AIR 27, 1950 SCR 88, AIR 1950 SUPREME COURT 27, 1963 MADLW 638 Author: Hiralal J.
Kania Bench: Hiralal J.
Kania, Saiyid Fazal Ali, Mehr Chand Mahajan, B.K. Mukherjea PETITIONER: A.K.
GOPALAN Vs. RESPONDENT: THE STATE OF MADRAS.UNION OF INDIA: INTERVENER. DATE OF JUDGMENT: 19/05/1950 BENCH: KANIA, HIRALAL J.
(CJ) BENCH: KANIA, HIRALAL J.
(CJ) FAZAL ALI, SAIYID SASTRI, M.
PATANJALI MAHAJAN, MEHR CHAND DAS, SUDHI RANJAN MUKHERJEA, B.K. CITATION: 1950 AIR 27 1950 SCR 88 CITATOR INFO : F 1951 SC 157 (21) F 1951 SC 270 (5,6) F 1951 SC 301 (10) F 1951 SC 332 (344) E 1952 SC 75 (45) RF 1952 SC 123 (6) F 1952 SC 181 (6,27,29,33) D 1952 SC 196 (16) RF 1952 SC 252 (106) R 1952 SC 366 (16) E&F 1952 SC 369 (90,93) F 1953 SC 451 (7) E&F 1954 SC 92 (5,39) RF 1954 SC 119 (15) RF 1954 SC 728 (17) R 1955 SC 41 (6) F 1956 SC 108 (6) R 1957 SC 688 (6,8,9) R 1958 SC 163 (7,8,9,25,26,33,36,38,41) APL 1958 SC 5

In [29]:
# [ ----- DATA EXPLORATION ----- ]
# i observe we can find very many more useful fields from this pdf. and hencewhy, i should parse
# all the documents and collect a list of all bold words, like CITATIONS JUDGEMENT etc and see
# the words that are present in all documents, that will give me a list of what further subdivisions
# are possible for each case
from collections import defaultdict
import string # to use string.punctuation which contains all filler used in english, useful, see
# print(string.punctuation)
all_cap_words = defaultdict(list)

for case in apex_raw:
  seen_in_case = defaultdict(int)
  for word in case["text"].split():
    w = word.strip(string.punctuation)
    if w.isupper() and len(w) > 1:
      seen_in_case[w] += 1
  for k,v in seen_in_case.items():
    all_cap_words[k].append(v)

for k,v in all_cap_words.items():
  if 50 < len(v) < 170:
    print(f"key :: {k} value :: {len(v)} values :: {v}")

'''
oh well lots of potential splits, but never occuring in ALL cases, so some/half the time, half the
columns/splits will go empty. Can do it in future, consider the columns where the count of word does
not change as well, like JUDGEMENT occurs two times in all cases, can be 3 its fine, but something like ALI
or SC occurs very variably 12 3 1 ... so i will drop them to get certain names that can be used as
subdivisions of text column, then will have to manually see how they are written to embed them as well
TODO
'''

key :: AIR value :: 96 values :: [3, 3, 3, 1, 1, 3, 3, 2, 2, 3, 1, 1, 3, 3, 3, 2, 2, 3, 3, 2, 1, 2, 2, 1, 3, 2, 3, 3, 1, 1, 3, 3, 2, 1, 3, 3, 1, 3, 2, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 3, 2, 3, 2, 3, 1, 2, 1, 3, 3, 2, 3, 2, 3, 1, 2, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 1, 2, 2, 1, 1, 2, 1, 2, 3, 3, 3, 3]
key :: SCR value :: 72 values :: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
key :: SUPREME value :: 77 values :: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
key :: COURT value :: 77 values :: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

Mate the time estimate is more than 10 hours, i HAVE to use multiprocessing here mate.

Also dam this text stuff useful to make notes instead of writing all that in comments. Definately underestimated notebook all this time.

Colab gives 1 worker, whats' the difference to using single threaded code then?!

In [ ]:
'''
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

Well so the idea is to make worker/threads and thread will go parse pdf for us, and we will just collect
data from our threads and write them orderly in jsonl to prevent conflicts, cheap way. Hence this
block is responsible for going to fetch the pdf, read it, and return us one line of our jsonl
'''
def process_pdf(args): # i can pass only 1 argument hence a tuple/struct
  year, filename, pdf_path, id = args
  pages = []

  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      text = page.extract_text()
      if text:
        pages.append(text)

  return {
    "id": id,
    "source": "Supreme Court",
    "title": filename,
    "text": strip_headers_footers(pages),
    "meta": {"year": year}
    }


In [ ]:
'''
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

Just like before we again parse all our folders and subfolders to read pdf(s)
But instead of reading directly we are making a list of 'tasks' we need to do
ie pfd(s) we need to read which contains, ofc, most importantly, the path to pdf
and some metadata that needs be stored in jsonl/datastructure of ours.
'''
import pdfplumber
import os
ROOT = "/content/sc_judgments/supreme_court_judgments"

tasks = [] # contains all files, weirdly enough i now know how many files i need to process
i = 1

for year in sorted(os.listdir(ROOT)):
  for file2 in sorted(os.listdir(f"{ROOT}/{year}")):
    tasks.append((
      year,
      file2,
      f"{ROOT}/{year}/{file2}",
      i
    ))
    i += 1


In [ ]:
"""
MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE MULTI CORE

"""
from concurrent.futures import ProcessPoolExecutor
import multiprocessing
import json

workers = max(1, multiprocessing.cpu_count() - 1)
print(f"the number of workers we have is :: {workers}")

total = len(tasks)
done = 0

# this (main process) is responsible for writing to pdf, so we open pdf
with open(path_to_json, "a", encoding="utf-8") as f:
  # and allocate our minions/threads their task they need to do
  with ProcessPoolExecutor(max_workers=workers) as executor:
    # and for each worker, we write back the result
    for result in executor.map(process_pdf, tasks):
      done += 1
      print(f"\rProcessed {done}/{total} PDFs :: year - {tasks[done-1][0]}")
      if result is not None:
        f.write(json.dumps(result, ensure_ascii=False))
        f.write("\n")
      if result % 100 == 0:
        !cp {path_to_json} /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl


the number of workers we have is :: 1
Processed 1/26688 PDFs :: year - 1950
Processed 2/26688 PDFs :: year - 1950
Processed 3/26688 PDFs :: year - 1950
Processed 4/26688 PDFs :: year - 1950
Processed 5/26688 PDFs :: year - 1950
Processed 6/26688 PDFs :: year - 1950
Processed 7/26688 PDFs :: year - 1950
Processed 8/26688 PDFs :: year - 1950
Processed 9/26688 PDFs :: year - 1950
Processed 10/26688 PDFs :: year - 1950
Processed 11/26688 PDFs :: year - 1950
Processed 12/26688 PDFs :: year - 1950
Processed 13/26688 PDFs :: year - 1950
Processed 14/26688 PDFs :: year - 1950
Processed 15/26688 PDFs :: year - 1950
Processed 16/26688 PDFs :: year - 1950
Processed 17/26688 PDFs :: year - 1950
Processed 18/26688 PDFs :: year - 1950
Processed 19/26688 PDFs :: year - 1950
Processed 20/26688 PDFs :: year - 1950
Processed 21/26688 PDFs :: year - 1950
Processed 22/26688 PDFs :: year - 1950
Processed 23/26688 PDFs :: year - 1950
Processed 24/26688 PDFs :: year - 1950
Processed 25/26688 PDFs :: year - 1

KeyboardInterrupt: 

This is actually cooler than multiprocessing cuz, ProcessPoolExecuter() actually gives us FULL FELEGED PROCESSES that are python process- each with own memory cputime and shit, and executor is responsible for handling it

In [ ]:
dataset_supereme_high_court = datasets.load_dataset("opennyaiorg/InJudgements_dataset", token="")


README.md:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

data/train-00000-of-00002-add4caaf8fbc6a(…):   0%|          | 0.00/150M [00:00<?, ?B/s]

data/train-00001-of-00002-09ac6bd45d6b36(…):   0%|          | 0.00/143M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11970 [00:00<?, ? examples/s]

In [93]:
# [LOAD BNS DATASET]
SAVE_PATH1 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns.json"

if FRESH_START:
  bns_raw = pandas.read_csv("/content/bns_data/bns_sections.csv")
  print(bns_raw.head())
  bns = []
  for i, entries in bns_raw.iterrows():
    temp = {
      'id' : i+1,
      'chapter': entries['Chapter'],
      'section': entries['Section'],
      'title': entries['Section _name'],
      'text': entries['Description']
    }
    bns.append(temp)
  print(bns[0])

  with open(SAVE_PATH1, 'w') as f:
    json.dump(bns, f)
else:
  with open(SAVE_PATH1, 'r') as f:
    bns = json.load(f)

In [94]:
print(type(bns))
def _clean_bns_data(data):
  sentances = data['text'].split('\r\n')
  data['text'] = '\n'.join(sentances)
  return data

for data in bns:
  data = _clean_bns_data(data)

print(bns[0]['text'])
print(bns[0])
# though one immediate problem i see is that it splits 1 2 3 4 then 4a 4b 4c to indipendent
# lines as well, maybe i should not split a b c stuff ?

<class 'list'>
(1) This Act may be called the Bharatiya Nyaya Sanhita, 2023.
(2) It shall come into force on such date1 as the Central Government may, by notification in the Official Gazette, appoint, and different dates may be appointed for different provisions of this Sanhita.
(3) Every person shall be liable to punishment under this Sanhita and not otherwise for every act or omission contrary to the provisions thereof, of which he shall be guilty within India.
(4) Any person liable, by any law for the time being in force in India, to be tried for an offence committed beyond India shall be dealt with according to the provisions of this Sanhita for any act committed beyond India in the same manner as if such act had been committed within India.
(5) The provisions of this Sanhita shall also apply to any offence committed by--
(a) any citizen of India in any place without and beyond India;
(b) any person on any ship or aircraft registered in India wherever it may be;
(c) any person in a

In [95]:
# [LOAD SUP-HI COURT DATASET]
SAVE_PATH2 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_courts.json"

# [THE TEXT DATA IS TOO NOISY WITH HTML TAGS]
def clean_html(text):
  import re
  text = re.sub(r"<span[^>]*>.*?</span>", " ", text, flags=re.DOTALL) # REMOVE 'EM HTML
  text = re.sub(r"<[^>]+>", " ", text) # REMOVE 'EM HTML
  text = re.sub(r"[=\-_.]{4,}", " ", text)

  return text

# print(dataset_supereme_high_court)
# print(dataset_supereme_high_court['train'][0])
if FRESH_START:
  high_courts_raw = dataset_supereme_high_court['train']
  # high_courts_raw[0]
  # print(high_courts_raw[0].keys())
  high_courts = []
  for i, cases in enumerate(high_courts_raw):
    temp = {
        'id' : i+1,
        'court_details' : f"{cases['Court_Type']} | {cases['Case_Type']} | {cases['Court_Name']}",
        'cited_by' : cases['Cited_by'],
        'title': cases['Titles'],
        # 'text' : " ".join(cases["Text"].split()) # too mych filler mate NEXT DAY REALIZATION : without spaces i have no idea where
        # a paragraph or context ends, so i end up making random chunks distributing meaning(s) very badly, keep it for now, remove
        # at chunking step instead
        'text' : clean_html(cases["Text"])
    }
    high_courts.append(temp)

  print(high_courts[0])
  with open(SAVE_PATH2, "w") as f:
    json.dump(high_courts, f)
else:
  with open(SAVE_PATH2, "r") as f:
    high_courts = json.load(f)

In [96]:
# [ ----- COnsider only high courts' decisions from this dataset ----- ]
# print(sum(1 for cases in high_courts if cases['court_details'].startswith('High')))
# print(sum(1 for cases in high_courts if cases['court_details'].startswith('Sup')))
# print(len(high_courts))

# well let us drop all supereme court cases from here and use it has high court dataset

high_courts = [high_court for high_court in high_courts if high_court['court_details'].startswith('High')]

# print(len(high_courts))
# high_courts[0]['text'] = "\n".join(line.strip() for line in high_courts[0]['text'].split('\n'))
# print(high_courts[0])
# print(high_courts[0]['text'])

# well my format demands paragraph spilts be defined by single \n and others be continous

In [97]:
import re
def _make_high_court_paragraphs(data):
  cleaned_lines = []
  # print(f":: toatal paragraphs made {len(data['text'].split('\n\n'))} :: ")
  length = 0
  for line in data['text'].split('\n\n'):
    line = " ".join(line.split())
    cleaned_lines.append(line)
    length = max(length, len(line))
  # print(f"longest paragraphs' length is {length}")
  data['text'] = '\n'.join(cleaned_lines)
  return data

print(high_courts[0])

for cases in high_courts:
  cases = _make_high_court_paragraphs(cases)

# print(high_courts[0])
print(high_courts[0:2])


{'id': 1, 'court_details': 'High_Court | Land&Property | Patna High Court - Orders', 'cited_by': 9, 'title': 'Maheshwar Mandal & Anr vs The State Of Bihar & Ors on 24 June, 2014', 'text': 'IN THE HIGH COURT OF JUDICATURE AT PATNA\n                  Civil Writ Jurisdiction Case No.1091 of 2013\n                                        With\n                      Interlocutory Application No. 1572 of 2013\n                                         In\n                  Civil Writ Jurisdiction Case No. 1091 of 2013\n \n1. Maheshwar Mandal S/O Late Bachchi Mandal\n2. Sanjay Mandal S/O Maheshwar Mandal, both resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria     Petitioners Versus\n1. The State of Bihar\n2. Divisional Commissioner, Purnea\n3. Land Reforms Deputy Collector, Forbesganj, Araria     Respoondents 1st Party\n4. Smt. Jashoda Devi, wife of Gosain Mandal, resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivisi

In [98]:
"""
[UNIFYING ALL DATA REPRESENTATIONS]
well i see no point in doing this, but apprearently it helps later on figuring what went wrong in
RAG design and extending it in future as well.
"""

new_bns = []
for entries in bns:
  temp = {
      'id' : f"BNS_{entries['id']}",
      'source' : "BNS",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
        'chapter' : entries['chapter'],
        'section' : entries['section']
        }
  }
  new_bns.append(temp)
bns = new_bns
del new_bns

new_court = []
for entries in high_courts:
  temp = {
      'id' : f"COURTS_{entries['id']}",
      'source' : "Courts",
      'title' : entries['title'],
      'text' : entries['text'],
      'meta' : {
          'cited_by' : entries['cited_by'],
          'court_details' : entries['court_details']
          }
  }
  new_court.append(temp)
court = new_court # yeh i changed name.
del new_court
del high_courts
print(court[0])
print("\n\n")
print(bns[0])

{'id': 'COURTS_1', 'source': 'Courts', 'title': 'Maheshwar Mandal & Anr vs The State Of Bihar & Ors on 24 June, 2014', 'text': 'IN THE HIGH COURT OF JUDICATURE AT PATNA Civil Writ Jurisdiction Case No.1091 of 2013 With Interlocutory Application No. 1572 of 2013 In Civil Writ Jurisdiction Case No. 1091 of 2013 1. Maheshwar Mandal S/O Late Bachchi Mandal 2. Sanjay Mandal S/O Maheshwar Mandal, both resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria Petitioners Versus 1. The State of Bihar 2. Divisional Commissioner, Purnea 3. Land Reforms Deputy Collector, Forbesganj, Araria Respoondents 1st Party 4. Smt. Jashoda Devi, wife of Gosain Mandal, resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria Respondent 2nd Party Appearance : For the Petitioners : Mr. Yogendra Mishra & Mr. Uma Kant Tiwary, Mr. Binod Kumar, Advocates For Respondent No. 4 : Ms. Kumari Ritambhara, Advocate For the S

In [99]:

print(court[0])
print("\n")
print(bns[0])
print("\n")
print(apex_raw[0])
print("\n\n")
print(court[0]['text'][:7000])
print("\n")
print(bns[0]['text'][:700])
print("\n")
print(apex_raw[0]['text'][:700])

{'id': 'COURTS_1', 'source': 'Courts', 'title': 'Maheshwar Mandal & Anr vs The State Of Bihar & Ors on 24 June, 2014', 'text': 'IN THE HIGH COURT OF JUDICATURE AT PATNA Civil Writ Jurisdiction Case No.1091 of 2013 With Interlocutory Application No. 1572 of 2013 In Civil Writ Jurisdiction Case No. 1091 of 2013 1. Maheshwar Mandal S/O Late Bachchi Mandal 2. Sanjay Mandal S/O Maheshwar Mandal, both resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria Petitioners Versus 1. The State of Bihar 2. Divisional Commissioner, Purnea 3. Land Reforms Deputy Collector, Forbesganj, Araria Respoondents 1st Party 4. Smt. Jashoda Devi, wife of Gosain Mandal, resident of Village Sahebganj, P.S., P.O. and Anchal- Narpatganj, Subdivision- Forbesganj, District Araria Respondent 2nd Party Appearance : For the Petitioners : Mr. Yogendra Mishra & Mr. Uma Kant Tiwary, Mr. Binod Kumar, Advocates For Respondent No. 4 : Ms. Kumari Ritambhara, Advocate For the S

In [15]:
print(court[2]['text'][:700])
print("\n\n")
print(bns[2]['text'][:700])


IN THE HIGH COURT OF JUDICATURE AT PATNA                          LPA 681 of 2010 1.Mt. Prabhawati widow of Late Madan Mishra, resident of village-   Pathakauli, P.S. Bagaha, District West Champaran2. Kamlawati Devi @ Kalawati Devi wife of Chhedi Singh3. Geeta Devi wife of Anirudh Singh   Both resident of Village- Narainpur, P.S. Bagaha, District West   Champaran4. Manorama Devi, wife of Ashok Kumar Singh, resident of village   Narainpur, P.S. Bagaha, District West Champaran5. Gayatri Devi wife of Surendra Kumar Rai, resident of Village   Rampur Tutulia, P.S. Shikarpur, District West Champaran6. Dhruva Prasad son of Late Beni Madhava Sah, resident of Village   Pathakauli, P.S. Bagaha, Distri



(1) Throughout this Sanhita every definition of an offence, every penal provision, and every Illustration of every such definition or penal provision, shall be understood subject to the exceptions contained in the Chapter entitled “General Exceptions”, though those exceptions are not repeated in

In [ ]:
# [OLD VERSION WONT BE USING THIS ONE]
# well now since i hesitantly unified representation of both(infuture three:: TODO TODO TODO TODO TODOOOOOOOOOOOOO) datasets' embeddings
# i can write a single function to break 'em all up in chunks and save the relevant info only.

def make_chunks(text, tokenizer, token_len=400, overlap_len=60):
    paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
    chunks = []
    current_tokens = []

    for para in paragraphs:
        para_tokens = tokenizer.encode(para)

        # if paragraph alone is too big, split it
        if len(para_tokens) > token_len:
            for i in range(0, len(para_tokens), token_len - overlap_len):
                chunks.append(para_tokens[i:i + token_len])
            current_tokens = []
            continue

        # if adding paragraph exceeds budget → flush
        if len(current_tokens) + len(para_tokens) > token_len:
            chunks.append(current_tokens)
            # overlap
            current_tokens = current_tokens[-overlap_len:] if overlap_len else []

        current_tokens.extend(para_tokens)

    if current_tokens:
        chunks.append(current_tokens)

    return chunks


In [100]:
'''
[well chunking it is mate]
assumed that my input will be a data structure which will contain a subfield "text" that i need
to chunk. And paragraphs will be seperated by \n so those are my assumptions on input
'''
import numpy as np
from sentence_transformers import SentenceTransformer

def split_long_paragraph(paragraph, tokenizer, max_tokens):
    words = paragraph.split()
    word_tokens = [len(tokenizer.encode(w, add_special_tokens=False)) for w in words]

    parts = []
    current = []
    current_len = 0

    for w, w_len in zip(words, word_tokens):
        current.append(w)
        current_len += w_len

        if current_len >= max_tokens:
            parts.append(" ".join(current))
            current = []
            current_len = 0

    if current:
        parts.append(" ".join(current))

    return parts

def _label_chunks(chunks, data):
  chunks_final = []
  for i, chunk in enumerate(chunks):
    temp = {
      'chunk_id' : f"{data['source']}_{data['id']}_{i}",
      'chunk_text' : "\n".join(chunk),
      'parent_id' : data['id'],
      'source' : data['source']
          # this should be enough to retireve original data if needed
    }
    chunks_final.append(temp)
  return chunks_final

def make_chunks(data, tokenizer, para_overlap=1, max_tokens=800, token_overlap=150):
  # split & clean paragraphs
  paragraphs = [p.strip() for p in data["text"].split("\n") if p.strip()]
  chunks = []
  chunk = []
  running_len = 0

  # precompute token lengths once
  # para_tokens = [len(tokenizer.encode(p)) for p in paragraphs]
  # print(f'precomputed tokens :: ')
  # total_exceeding = sum(1 for paragraph_len in para_tokens if paragraph_len > 255)
  # print(f" :: total exceeding :: {total_exceeding}")
  expanded_paragraphs = []

  for p in paragraphs:
    p_len = len(tokenizer.encode(p, add_special_tokens=False))

    if p_len > max_tokens:
      parts = split_long_paragraph(p, tokenizer, max_tokens)
      for part in parts:
        part_len = len(tokenizer.encode(part, add_special_tokens=False))
        expanded_paragraphs.append((part, part_len))
    else:
      expanded_paragraphs.append((p, p_len))

  paragraphs = [p for p, _ in expanded_paragraphs]
  para_tokens = [l for _, l in expanded_paragraphs]

  # total_exceeding = sum(1 for paragraph_len in para_tokens if paragraph_len > 255)
  # print(f" :: total exceeding :: {total_exceeding}")

  for i, paragraph in enumerate(paragraphs):
    p_len = para_tokens[i]

    # handle absurdly long single paragraph
    # if p_len > max_tokens:
    #   continue  # or sentence-split later

    # if adding paragraph would overflow
    if running_len + p_len > max_tokens:
      # save current chunk
      if chunk:
          chunks.append(chunk.copy())

      # start new chunk with overlap
      chunk = []
      running_len = 0

      if para_overlap > 0:
        for j in range(max(0, i - para_overlap), i):
          overlap_len = para_tokens[j]
          if running_len + overlap_len > token_overlap:
            break
          chunk.append(paragraphs[j])
          running_len += overlap_len

    # add current paragraph
    chunk.append(paragraph)
    running_len += p_len

  # save final chunk
  if chunk:
    chunks.append(chunk.copy())

  return _label_chunks(chunks, data)
# make_chunks(apex_raw[0], SentenceTransformer('all-MiniLM-L6-v2').tokenizer)
# make_chunks(court[0], SentenceTransformer('all-MiniLM-L6-v2').tokenizer)
# make_chunks(bns[0], SentenceTransformer('all-MiniLM-L6-v2').tokenizer)

In [101]:
!pip install tqdm
from tqdm import tqdm

In [11]:
model_faiss = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

In [8]:
# !pip install sentence_transformers
import sentence_transformers
import json
FRESH_START = False

In [9]:
model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = model.tokenizer
chunks_bns = []
chunks_high = []
chunks_apex = []
if FRESH_START:
  print(f' :: currently doing apex :: ')
  for cases in tqdm(apex_raw):
    labeled_chunk = make_chunks(cases, tokenizer, 1, 800, 50)
    chunks_apex += labeled_chunk
  with open('apex_chunked.json', 'w', encoding='utf-8') as f:
    json.dump(chunks_apex, f, ensure_ascii=False)
  !cp /content/apex_chunked.json /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_chunked.json

  print(f" :: currently doing high_courts :: ")
  for cases in tqdm(court):
    labeled_chunk = make_chunks(cases, tokenizer, 1, 800, 50)
    chunks_high += labeled_chunk
  with open('high_chunked.json', 'w', encoding='utf-8') as f:
    json.dump(chunks_high, f, ensure_ascii=False)
  !cp /content/high_chunked.json /content/drive/MyDrive/colab_stuff/court_hearings_rag/high_chunked.json

  print(f':: currently doing BNS :: ')
  for cases in tqdm(bns):
    labeled_chunk = make_chunks(cases, tokenizer, 0, 800, 0)
    chunks_bns += labeled_chunk
  with open('bns_chunked.json', 'w', encoding='utf-8') as f:
    json.dump(chunks_bns, f, ensure_ascii=False)
  !cp /content/bns_chunked.json /content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_chunked.json
else:
  with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_chunked.json', 'r', encoding='utf-8') as f:
    chunks_apex = json.load(f)

  with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_chunked.json', 'r', encoding='utf-8') as f:
    chunks_high = json.load(f)

  with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_chunked.json', 'r', encoding='utf-8') as f:
    chunks_bns = json.load(f)



In [5]:
print(len(chunks_apex))
print(len(chunks_high))
print(len(chunks_bns))
print(type(chunks_apex))
print(chunks_apex[0])
print(chunks_apex[1])

258124
157780
376
<class 'list'>
{'chunk_id': 'Supreme Court_1_0', 'chunk_text': 'A.K.\nGopalan vs The State Of Madras.Union Of India: ...\non 19 May, 1950 Equivalent citations: 1950 AIR 27, 1950 SCR 88, AIR 1950 SUPREME COURT 27, 1963 MADLW 638 Author: Hiralal J.\nKania Bench: Hiralal J.\nKania, Saiyid Fazal Ali, Mehr Chand Mahajan, B.K. Mukherjea PETITIONER: A.K.\nGOPALAN Vs. RESPONDENT: THE STATE OF MADRAS.UNION OF INDIA: INTERVENER. DATE OF JUDGMENT: 19/05/1950 BENCH: KANIA, HIRALAL J.\n(CJ) BENCH: KANIA, HIRALAL J.\n(CJ) FAZAL ALI, SAIYID SASTRI, M.', 'parent_id': 1, 'source': 'Supreme Court'}
{'chunk_id': 'Supreme Court_1_1', 'chunk_text': 'PATANJALI MAHAJAN, MEHR CHAND DAS, SUDHI RANJAN MUKHERJEA, B.K. CITATION: 1950 AIR 27 1950 SCR 88 CITATOR INFO : F 1951 SC 157 (21) F 1951 SC 270 (5,6) F 1951 SC 301 (10) F 1951 SC 332 (344) E 1952 SC 75 (45) RF 1952 SC 123 (6) F 1952 SC 181 (6,27,29,33) D 1952 SC 196 (16) RF 1952 SC 252 (106) R 1952 SC 366 (16) E&F 1952 SC 369 (90,93) F 1953 

In [ ]:
# CHUNKING LOGIC 3 ---->
# Problem - most of our so called paragraphs are bound to exceed token length of most models
# so we need to break them to sub paragraphs.
# REQUIREMENT - To make chunker that can chunks paragraphs and sub paragraphs, in a way that
# the overlapping is possible with sub paragraphs if not paragraphs itself.
# IDEA - even if paragraph is too long, sentances themselves aren't, so we split paragraph to
# sentances
# DESPERATION - for now i will just break them randomly not specifically at sentance boundary
# to see it all works at least

In [ ]:
'''
hear the news, faiss returns integer index of what was used to embed, so, embedding_list[i]  <same order>  metadata_list[i]
'''


In [3]:
'''
make embedding and select model
'''
model_name = 'all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = sentence_transformers.SentenceTransformer(model_name, device=device)
# remember that faiss is cpu first, need to move everything to gpu manually

def embed(model, embedding_name, chunks):
  # model = sentence_transformers.SentenceTransformer(model_name, device=device)
  path = f'/content/embedding_{embedding_name}.npy'
  print(f'currently embedding za {embedding_name}')

  if not FRESH_START:
    # model.to(device) not adviced, do at creation, else things break
    chunk_texts = [chunk['chunk_text'] for chunk in chunks]
    embeddings = model.encode(
      chunk_texts,
      batch_size = 64,
      convert_to_numpy=True, # remember faiss numpy
      show_progress_bar=True,
      normalize_embeddings=True,
      device=device
      )
    np.save(path, embeddings)
    !cp {path} /content/drive/MyDrive/colab_stuff/court_hearings_rag/{embedding_name}.npy
  else:
    embeddings = np.load(path)
  return embeddings
print(torch.cuda.is_available())

True


In [5]:
if FRESH_START:
  embedding_bns = embed(model, 'bns_embed', chunks_bns)
  embedding_apex = embed(model, 'apex_embed', chunks_apex)
  embedding_high = embed(model, 'high_embed', chunks_high)
else:
  embedding_bns = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_embed.npy')
  embedding_apex = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_embed.npy')
  embedding_high = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_embed.npy')


In [7]:
cuda = torch.cuda.is_available()
cuda = False # fuk it, i need cpu ones for portability
# model takes all of gpu, need to keep faiss to cpu
!pip install faiss-gpu-cu12
import faiss

dim = embedding_bns.shape[1]

index_bns = faiss.IndexFlatIP(dim)
index_apex = faiss.IndexFlatIP(dim)
index_high = faiss.IndexFlatIP(dim)

if cuda:
  bns_gpu = faiss.StandardGpuResources()
  apex_gpu = faiss.StandardGpuResources()
  high_gpu = faiss.StandardGpuResources()
  index_bns = faiss.index_cpu_to_gpu(bns_gpu, 0, index_bns) # cuda is assumed, just pass 0 as in cuda:0
  index_apex = faiss.index_cpu_to_gpu(apex_gpu, 0, index_apex)
  index_high = faiss.index_cpu_to_gpu(high_gpu, 0, index_high)

index_bns.add(embedding_bns)
index_apex.add(embedding_apex)
index_high.add(embedding_high)

# print(len(embedding_apex))

In [8]:
faiss.write_index(index_bns, "/content/index_bns.faiss")
faiss.write_index(index_apex, "/content/index_apex.faiss")
faiss.write_index(index_high, "/content/index_high.faiss")

In [9]:
!cp /content/index_bns.faiss /content/drive/MyDrive/colab_stuff/court_hearings_rag/index_bns.faiss
!cp /content/index_apex.faiss /content/drive/MyDrive/colab_stuff/court_hearings_rag/index_apex.faiss
!cp /content/index_high.faiss /content/drive/MyDrive/colab_stuff/court_hearings_rag/index_high.faiss

In [10]:
!pip install faiss-cpu
import faiss

index_bns = faiss.read_index('/content/drive/MyDrive/colab_stuff/court_hearings_rag/index_bns.faiss')
index_apex = faiss.read_index('/content/drive/MyDrive/colab_stuff/court_hearings_rag/index_apex.faiss')
index_high = faiss.read_index('/content/drive/MyDrive/colab_stuff/court_hearings_rag/index_high.faiss')

In [27]:
def search_info(query, model, metadata, top_k=5):
  # first use the same sentance embedding model to make query database compatible
  query_vector = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)

  sim_1, idx_1 = index_bns.search(query_vector, top_k)
  sim_2, idx_2 = index_apex.search(query_vector, top_k)
  sim_3, idx_3 = index_high.search(query_vector, top_k )

  bns, apex, high = metadata
  print(len(query))
  for q in range(len(query)):
    for i in range(top_k):
      # even if we don't use list of queries, the faiss treats it as such and returns
      # similarity score and index in 2d form as in similarity[i][j] where i corrosponds
      # to each query in input, and j is top j matches, so i decided to keep list of queries
      # format itself, instead of calling in loop this function for i queries, just pass
      # a list of i queries, thats' how its built afterall.
      print(f'{i+1}th most similar to query in bns with similarity :: {sim_1[q][i]} is :: \n{bns[idx_1[q][i]]}')
      print(f'{i+1}th most similar to query in apex with similarity :: {sim_2[q][i]} is :: \n{apex[idx_2[q][i]]}')
      print(f'{i+1}th most similar to query in high with similarity :: {sim_3[q][i]} is :: \n{high[idx_3[q][i]]}')


query = [
    "What's the reason of so many pending Cases accross india, and rights for minorities not getting treated equally?",
    "What are the fundamental rights of people in India, and What kind of government structure India is based upon?",
    "Explain Preamble.",
    "What is the meaning of 'secular' in Indian Law?"
  ]

metadata = (chunks_bns, chunks_apex, chunks_high)
search_info(query, model, metadata, 2)

4
1th most similar to query in bns with similarity :: 0.40111780166625977 is :: 
{'chunk_id': 'BNS_BNS_197_0', 'chunk_text': '(1) Whoever, by words either spoken or written or by signs or by visible representations or through electronic communication or otherwise,—\n(a) makes or publishes any imputation that any class of persons cannot, by reason of their being members of any religious, racial, language or regional group or caste or community, bear true faith and allegiance to the Constitution of India as by law established or uphold the sovereignty and integrity of India; or\n(b) asserts, counsels, advises, propagates or publishes that any class of persons shall, by reason of their being members of any religious, racial, language or regional group or caste or community, be denied, or deprived of their rights as citizens of India; or\n(c) makes or publishes any assertion, counsel, plea or appeal concerning the obligation of any class of persons, by reason of their being members of any 

In [13]:
!pip install -U transformers accelerate bitsandbytes

from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


In [14]:
import transformers
import torch
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto" # automatically does cpu/gpu
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_fast=True # use rust backend when available
)
model.eval() # it is all pytorch afterall.
print(model.device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

cuda:0


In [18]:
# first pass

question = 'How has Article 21 been interpreted by the Supreme Court over time?'

prompt = """<s>[INST]
    You are a query rewriting system for semantic search.

    Task:
    Rewrite the user's question into a neutral, explicit search query that would help retrieve
    relevant legal or regulatory documents from a vector database.

    Rules:
    - Do NOT answer the question
    - Do NOT give advice
    - Do NOT mention uncertainty
    - Do NOT add explanations
    - Rewrite as a single declarative sentence
    - Use general legal terminology where appropriate
    - Remove casual language

    User question:
    {user_question}
    [/INST]""".format(user_question = question)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False
    )

reply = tokenizer.decode(output[0], skip_special_tokens=True)
print(reply)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]
    You are a query rewriting system for semantic search.

    Task:
    Rewrite the user's question into a neutral, explicit search query that would help retrieve
    relevant legal or regulatory documents from a vector database.

    Rules:
    - Do NOT answer the question
    - Do NOT give advice
    - Do NOT mention uncertainty
    - Do NOT add explanations
    - Rewrite as a single declarative sentence
    - Use general legal terminology where appropriate
    - Remove casual language

    User question:
    How has Article 21 been interpreted by the Supreme Court over time?
    [/INST] Retrieve documents containing the term "Article 21" and "Supreme Court" and "interpretation" or "judicial interpretation" or "case law" or "legal precedent".


In [19]:
def faiss_search(query, model, metadata, top_k=2):
    # this function, this time, is written for 1 query at a time
    query_encoded = model.encode([query], normalize_embeddings=True, convert_to_numpy=True)
    documents = []
    documents.append(index_bns.search(query_encoded, top_k))
    documents.append(index_apex.search(query_encoded, top_k))
    documents.append(index_high.search(query_encoded, top_k))

    def _build_string_for_llm(doc, document):
        sim_i, idx_i = doc
        fetched = []
        for i in range(top_k):
            # (i)th similar
            temp = f'''
                confidence : {sim_i[0][i]},
                {metadata[document][idx_i[0][i]]}
            '''
            fetched.append(temp)
        return "".join(fetched)

    return "".join([_build_string_for_llm(doc, i) for i, doc in enumerate(documents)])
metadata = (chunks_bns, chunks_apex, chunks_high)
print(faiss_search(reply, model_faiss, metadata, 1))


                confidence : 0.2989271283149719,
                {'chunk_id': 'BNS_BNS_229_0', 'chunk_text': '(1) Whoever intentionally gives false evidence in any stage of a judicial proceeding, or fabricates false evidence for the purpose of being used in any stage of a judicial proceeding, shall be punished with imprisonment of either description for a term which may extend to seven years, and shall also be liable to fine which may extend to ten thousand rupees.\n(2) Whoever intentionally gives or fabricates false evidence in any case other than that referred to in sub-section (1), shall be punished with imprisonment of either description for a term which may extend to three years, and shall also be liable to fine which may extend to five thousand rupees.\nExplanation 1.—A trial before a Court-martial is a judicial proceeding.\nExplanation 2.—An investigation directed by law preliminary to a proceeding before a Court, is a stage of a judicial proceeding, though that investigation m

In [20]:
documents = faiss_search(reply, model_faiss, metadata)
prompt = """<s>[INST]
Query:
You are a legal assistant, you are asked a question and given
relevent document snippets with confidence score indicating how
relevent can document be to the question. You should read the
full document to find relevent information then answer user query
based off of it.

Your reply must be
Concise
Easy to understand
Based on documents

Question:
{user_question}

Documents:
{documents}
[/INST]""".format(user_question = question, documents = documents)

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.74 GiB. GPU 0 has a total capacity of 14.74 GiB of which 2.66 GiB is free. Process 412936 has 12.08 GiB memory in use. Of the allocated memory 8.42 GiB is allocated by PyTorch, and 3.53 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)